In [1]:
import os; work_dir = "../../"; os.chdir(work_dir); work_dir = os.getcwd(); print("Current working directory:", work_dir)

Current working directory: /home/wilfried/Desktop/python_workspace/PCD_SM-Fluid-Pred


In [2]:
from results.data import multiple_GT_loading, make_results_dicts, load_multiple_results
from results.variables import method_strings, regularization, sde_names, test_cases
from results.JHTDB_128tra.variables import mean, std

IDs = ['y2025_m04_d24_11h_11m_40s', 'y2025_m04_d09_23h_16m_29s', 'y2025_m04_d28_08h_53m_22s',
       'y2025_m04_d24_00h_29m_31s', 'y2025_m04_d24_03h_35m_40s', 'y2025_m04_d24_06h_14m_58s']


dataKwargs = dict(test_cases=test_cases, means=mean, stds=std, dataset_name=(dataset_name:="JHTDB"))
strKwargs = dict(any_method_string=method_strings[0], any_ID=IDs[0], any_version=1)

GTs, fields_name = multiple_GT_loading(**strKwargs, **dataKwargs); print(fields_name)
GT_1, GT_2, GT_3 = GTs[1], GTs[2], GTs[3]

# Form: test_set = {"GT": {"pred": GT_1},
# "regFalse": {sde: {} for sde in sde_names}, "regTrue": {sde: {} for sde in sde_names}}
empty_test_sets = make_results_dicts(GTs, regularization, sde_names, test_cases)  # test_set_1, ... =

['velocity_x', 'velocity_y', 'density', 'pressure', 'mach']


In [3]:
from results.JHTDB_128tra.variables import fields_name; print(fields_name)
import numpy as np

fields_name_latex = ['$u$', '$v$', '$\\rho$', '$P$']

['velocity $u$', 'velocity $v$', 'density $\\rho$', 'pressure $P$', 'mach $Ma$']


In [4]:
versions = [1,2,3,4,5]

dataKwargs = dict(means=mean, stds=std, dataset_name=dataset_name, mask=(mask:=GTs["mask"]))
casesKwargs = dict(empty_test_sets=empty_test_sets, versions=versions, test_cases=test_cases)
test_sets = load_multiple_results(method_strings, IDs, regularization, sde_names, **casesKwargs, **dataKwargs)

 # e.g. test_set_1["regTrue"]["vpsde"]["pred"][42]
test_set_1, test_set_2, test_set_3 = test_sets[1], test_sets[2], test_sets[3]
num_fields = len(fields_name_latex)-1

In [5]:
from results.operations import GT_measures, pred_measures


n_bins = 200

GT_measures(test_set_1, fields_name, n_bins, dataset_name=dataset_name)
GT_measures(test_set_2, fields_name, n_bins, dataset_name=dataset_name)
GT_measures(test_set_3, fields_name, n_bins, dataset_name=dataset_name)

for version in versions:
    # e.g. test_set_1["regTrue"]["vpsde"]["spatial_histogram"][42][2], the 3rd field
    # After key measure is input, input [version] except for GT
    pred_measures(test_set_1, version, regularization, sde_names, fields_name, n_bins, dataset_name=dataset_name)
    pred_measures(test_set_2, version, regularization, sde_names, fields_name, n_bins, dataset_name=dataset_name)
    pred_measures(test_set_3, version, regularization, sde_names, fields_name, n_bins, dataset_name=dataset_name)

In [6]:
import numpy as np
import math
from itertools import product

def _flatten(d):
    """Return a 1‑D np.array no matter if *d* is list/array or dict."""
    if isinstance(d, dict):
        return np.concatenate([np.ravel(v) for v in d.values()])
    return np.ravel(d)

def _mean_var(d):
    d = _flatten(d)
    return d.mean(), d.var(ddof=1)

def _scientific_exponent(vals):
    mags = [abs(v) for v in vals if v != 0]
    if not mags:
        return 0
    exp = int(math.floor(math.log10(max(mags))))
    # keep mantissas in [0.01,100)
    if max(mags) / 10**exp < 0.1:
        exp -= 1
    return exp

def build_latex_table(
        test_cases,
        sde_names,                     # iterable
        regularization,                # iterable of True/False (or anything hashable)
        measures,                      # iterable of keys
        measure_labels,                # len == len(measures)
        versions,                      # iterable
        *,
        sde_labels=None,               # mapping or iterable aligned w/ sde_names
        regu_labels=None,              # mapping or iterable aligned w/ regularization
        highlight_lowest=None,         # list[bool] same length as measures
        num_decimals=2):

    # ------- label helpers -------
    if sde_labels is None:
        sde_labels = {k: str(k) for k in sde_names}
    elif not isinstance(sde_labels, dict):
        sde_labels = {k: lbl for k, lbl in zip(sde_names, sde_labels)}

    if regu_labels is None:
        regu_labels = {k: str(k) for k in regularization}
    elif not isinstance(regu_labels, dict):
        regu_labels = {k: lbl for k, lbl in zip(regularization, regu_labels)}

    if highlight_lowest is None:
        highlight_lowest = [True] * len(measures)

    # ---------- aggregation ----------
    result = {}
    for reg, sde in product(regularization, sde_names):
        rows = []
        for m in measures:
            cols = []
            for tc in test_cases:
                data = [_flatten(tc[reg][sde][m][v]) for v in versions]
                data = np.concatenate(data)
                cols.append(_mean_var(data))
            rows.append(cols)
        result[(reg, sde)] = rows

    # ---------- exponents per measure ----------
    exponents = []
    for mi, m in enumerate(measures):
        vals = [result[(r, s)][mi][ti][0]
                for r in regularization for s in sde_names
                for ti in range(len(test_cases))]
        exponents.append(_scientific_exponent(vals))

    # ---------- collect scaled means (for boldface) ----------
    n_cols = len(test_cases) * len(measures)
    column_values = [[] for _ in range(n_cols)]
    for reg, sde in product(regularization, sde_names):
        for mi in range(len(measures)):
            exp = exponents[mi]
            for ti in range(len(test_cases)):
                mean = result[(reg, sde)][mi][ti][0] / 10**exp
                column_values[mi*len(test_cases)+ti].append(mean)

    # choose minima or maxima per measure
    column_best = []
    for mi in range(len(measures)):
        for ti in range(len(test_cases)):
            col_idx = mi*len(test_cases)+ti
            if highlight_lowest[mi]:
                column_best.append(min(column_values[col_idx]))
            else:
                column_best.append(max(column_values[col_idx]))

    # ---------- LaTeX assembly ----------
    def fmt(mean, var, exp):
        mean_s = f"{mean/10**exp:.{num_decimals}f}"
        var_s  = f"{var/10**exp:.{num_decimals}f}"
        return f"{mean_s} $\\pm$ {var_s}"

    col_format = "|c|c|" + "|".join("c"*len(test_cases) for _ in measures) + "|"

    lines = [f"\\begin{{tabular}}{{{col_format}}}",
             "\\hline",
             "\\textbf{SDE type} & \\textbf{Regularization} & " +
             " & ".join(
                 f"\\multicolumn{{{len(test_cases)}}}{{c|}}{{{lbl} ($10^{{{exp}}}$)}}"
                 for lbl, exp in zip(measure_labels, exponents)
             ) + " \\\\"]

    # clines (first block already done)
    running = 3
    for i in range(len(measures)):
        start = running
        end   = running + len(test_cases) - 1
        lines.append(f"\\cline{{{start}-{end}}}")
        running += len(test_cases)

    # table body
    for reg in regularization:
        for sde in sde_names:
            row_cells = []
            for mi in range(len(measures)):
                exp = exponents[mi]
                for ti in range(len(test_cases)):
                    mean, var = result[(reg, sde)][mi][ti]
                    scaled_mean = mean / 10**exp
                    col_idx = mi*len(test_cases)+ti
                    cell = fmt(mean, var, exp)
                    if math.isclose(scaled_mean, column_best[col_idx]):
                        cell = f"\\textbf{{{cell}}}"
                    row_cells.append(cell)

            lines.append(f"{sde_labels[sde]} & {regu_labels[reg]} & "
                         + " & ".join(row_cells) + " \\\\")
        lines.append("\\hline")

    lines.append("\\end{tabular}")
    return "\n".join(lines)

In [7]:
test_cases = [test_set_1, test_set_2, test_set_3]


# latex = build_latex_table(
#     test_cases       = test_cases,
#     sde_names        = ["vpsde", "subvpsde", "vesde"],
#     regularization   = regularization,
#     measures         = [r"MSE $\downarrow$", r"pearson $\uparrow$", "wasserstein_distance", "KL_divergence"],
#     measure_labels   = ["MSE", "pearson", "W1", "KL"],
#     versions         = versions
# )


latex = build_latex_table(
    test_cases       = test_cases,
    sde_names        = ["vpsde", "subvpsde", "vesde"],
    regularization   = regularization,
    measures         = ["MSE", "pearson", "RE", "KL_divergence"],
    measure_labels   = ["MSE $\downarrow$", "pearson $\\uparrow$", "RE $\downarrow$", "KL"],
    versions         = versions,
    sde_labels       = ["VP","sub‑VP","VE"],
    regu_labels      = {regularization[0]:"False", regularization[1]:"True"},
    highlight_lowest = [True, False, True, True]   # MSE ↓, FID ↑, PSNR ↓
)

print(latex)

\begin{tabular}{|c|c|ccc|ccc|ccc|ccc|}
\hline
\textbf{SDE type} & \textbf{Regularization} & \multicolumn{3}{c|}{MSE $\downarrow$ ($10^{-3}$)} & \multicolumn{3}{c|}{pearson $\uparrow$ ($10^{-1}$)} & \multicolumn{3}{c|}{RE $\downarrow$ ($10^{-1}$)} & \multicolumn{3}{c|}{KL ($10^{0}$)} \\
\cline{3-5}
\cline{6-8}
\cline{9-11}
\cline{12-14}
VP & False & 2.52 $\pm$ 0.02 & 1.49 $\pm$ 0.01 & 2.82 $\pm$ 0.03 & 9.90 $\pm$ 0.00 & 9.93 $\pm$ 0.00 & 9.88 $\pm$ 0.00 & 1.08 $\pm$ 0.27 & 0.86 $\pm$ 0.19 & \textbf{1.12 $\pm$ 0.40} & 5.01 $\pm$ 83.08 & 1.98 $\pm$ 11.07 & 5.25 $\pm$ 93.89 \\
sub‑VP & False & 1.77 $\pm$ 0.01 & 1.26 $\pm$ 0.01 & 4.94 $\pm$ 0.06 & 9.92 $\pm$ 0.00 & 9.94 $\pm$ 0.00 & 9.78 $\pm$ 0.00 & \textbf{0.95 $\pm$ 0.18} & \textbf{0.72 $\pm$ 0.15} & 1.64 $\pm$ 0.70 & 5.02 $\pm$ 88.55 & \textbf{1.88 $\pm$ 9.80} & 5.34 $\pm$ 99.47 \\
VE & False & 7.45 $\pm$ 0.09 & 8.67 $\pm$ 0.14 & 8.17 $\pm$ 0.11 & 9.67 $\pm$ 0.01 & 9.61 $\pm$ 0.01 & 9.64 $\pm$ 0.01 & 2.06 $\pm$ 0.83 & 2.28 $\pm$ 1.24 & 